In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -yq
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -yq
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -yq
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -yq
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -yq
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -yq

In [ ]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

from fastai.vision.all import *

# DCM to PNG

In [ ]:
sample_sub = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")
test_dicom_files = get_files("../input/siim-covid19-detection/test/")
assert len(set(o.parent.parent.name for o in test_dicom_files)) + len(test_dicom_files) == sample_sub.shape[0]

In [ ]:
DEBUG  = True

In [ ]:
if len(test_dicom_files) == 1263 and DEBUG: test_dicom_files = test_dicom_files[:32]
test_image_ids = L(map(lambda o: o.stem, test_dicom_files)); len(test_image_ids)

In [ ]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
RESIZE = 512
save_dir = Path(f'/kaggle/working/test_images_{RESIZE}')
os.makedirs(save_dir, exist_ok=True)

imageid2size = {} #save original shape
studyid2imageids = defaultdict(list) # save study ids to image ids for study level preds

for file in progress_bar(test_dicom_files):
    # set keep_ratio=True to have original aspect ratio
    image_id = file.stem
    study_id = file.parent.parent.name
    studyid2imageids[study_id].append(image_id)
    xray = read_xray(file)
    imageid2size[image_id] = xray.shape
    im = resize(xray, size=RESIZE)  
    im.save(save_dir/f"{image_id}.png")

In [ ]:
# RESIZE = 384
# save_dir = Path(f'/kaggle/working/test_images_{RESIZE}')
# os.makedirs(save_dir, exist_ok=True)

# for file in progress_bar(test_dicom_files):
#     # set keep_ratio=True to have original aspect ratio
#     image_id = file.stem
#     xray = read_xray(file)
#     im = resize(xray, size=RESIZE)  
#     im.save(save_dir/f"{image_id}.png")

In [ ]:
image_id2study_id = {o.stem:o.parent.parent.name for o in test_dicom_files}; len(image_id2study_id)

# Detection (Detectron2)

In [ ]:
!nvcc --version

In [ ]:
!pip install -q ../input/detectron2-whl/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar # pycocotools
!pip install -q ../input/detectron2-whl/omegaconf-2.0.6-py3-none-any.whl # omegaconf
!pip install -q ../input/detectron2-whl/iopath-0.1.8-py3-none-any.whl # iopath
!pip install -q ../input/detectron2-whl/fvcore-master/fvcore-master # fvcore
!pip install -q ../input/detectron2-whl/detectron2-0.4cu102-cp37-cp37m-linux_x86_64.whl --no-deps

<a id="pred_method"></a>
## Prediction method implementations

Basically we don't need to implement neural network part, `detectron2` already implements famous architectures and provides its pre-trained weights. We can finetune these pre-trained architectures.

These models are summarized in [MODEL_ZOO.md](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md).

In this competition, we need object detection model, I will choose [R50-FPN](https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml) for this kernel.

## Data preparation

`detectron2` provides high-level API for training custom dataset.

To define custom dataset, we need to create **list of dict** where each dict contains following:

 - file_name: file name of the image.
 - image_id: id of the image, index is used here.
 - height: height of the image.
 - width: width of the image.
 - annotation: This is the ground truth annotation data for object detection, which contains following
     - bbox: bounding box pixel location with shape (n_boxes, 4)
     - bbox_mode: `BoxMode.XYXY_ABS` is used here, meaning that absolute value of (xmin, ymin, xmax, ymax) annotation is used in the `bbox`.
     - category_id: class label id for each bounding box, with shape (n_boxes,)

`get_vinbigdata_dicts` is for train dataset preparation and `get_vinbigdata_dicts_test` is for test dataset preparation.

### Prepare Data

In [ ]:
import pickle
from pathlib import Path
from typing import Optional

import cv2
import numpy as np
import pandas as pd
from detectron2.structures import BoxMode
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold

In [ ]:
test_image_ids[:3]

In [ ]:
def get_test_datasetdicts(test_image_ids, resized_height=512,resized_width=512):

    dataset_dicts = []
    for image_id in progress_bar(test_image_ids):
        record = {}

        record["file_name"] = f"/kaggle/working/test_images_512/{image_id}.png"
        record["image_id"] = image_id
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
datasetdicts = get_test_datasetdicts(test_image_ids)
datasetdicts[:3]

Methods for prediction for this competition

In [ ]:
# Methods for prediction for this competition
from math import ceil
from typing import Any, Dict, List

import cv2
import detectron2
import numpy as np
from numpy import ndarray
import pandas as pd
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode, Visualizer
from tqdm import tqdm


def format_pred(labels: ndarray, boxes: ndarray, scores: ndarray) -> str:
    pred_strings = []
#     class_names = ['opacity']
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
#         pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
        pred_strings.append(f"opacity {score} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)


def predict_batch(predictor: DefaultPredictor, im_list: List[ndarray]) -> List:
    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        inputs_list = []
        for original_image in im_list:
            # Apply pre-processing to image.
            if predictor.input_format == "RGB":
                # whether the model expects BGR inputs or RGB
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            # Do not apply original augmentation, which is resize.
            # image = predictor.aug.get_transform(original_image).apply_image(original_image)
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
        predictions = predictor.model(inputs_list)
        return predictions
    
def predict_batch(predictor: DefaultPredictor, im_list: List[ndarray]) -> List:
    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        inputs_list = []
        inputs_list_tta = []
        for original_image in im_list:
            # Apply pre-processing to image.
            if predictor.input_format == "RGB":
                # whether the model expects BGR inputs or RGB
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            # Do not apply original augmentation, which is resize.
            # image = predictor.aug.get_transform(original_image).apply_image(original_image)
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
            inputs_tta = {"image": image.flip(dims=[2]), "height": height, "width": width}
            inputs_list_tta.append(inputs_tta)
        
        predictions     = predictor.model(inputs_list)
        predictions_tta = predictor.model(inputs_list_tta)
        return predictions, predictions_tta

In [ ]:
# --- utils ---
from pathlib import Path
from typing import Any, Union

import yaml


def save_yaml(filepath: Union[str, Path], content: Any, width: int = 120):
    with open(filepath, "w") as f:
        yaml.dump(content, f, width=width)


def load_yaml(filepath: Union[str, Path]) -> Any:
    with open(filepath, "r") as f:
        content = yaml.full_load(f)
    return content


In [ ]:
# --- configs ---
thing_classes = ["opacity"]
category_name_to_id = {class_name: index for index, class_name in enumerate(thing_classes)}
category_name_to_id

This `Flags` class is to manage experiments. I will tune these parameters through the competition to improve model's performance.

In [ ]:
# --- flags ---
from dataclasses import dataclass, field
from typing import Dict


@dataclass
class Flags:
    # General
    debug: bool = True
    outdir: str = "results/det"

    # Data config
    imgdir_name: str = "siim-covid19-resized-to-512px-png"
    split_mode: str = "all_train"  # all_train or kfold
    fold_num: int = 0
    use_negative_class: bool = False
    oversample_classes = False
    seed: int = 111
    train_data_type: str = "original"  # original or wbf
    # Training config
    iter: int = 10000
    checkpoint_iter: int = 2000
    ims_per_batch: int = 24 # images per batch, this corresponds to "total batch size"
    num_workers: int = 4
    lr_scheduler_name: str = "WarmupMultiStepLR"  # WarmupMultiStepLR (default) or WarmupCosineLR
    base_lr: float = 0.00025
    roi_batch_size_per_image: int = 512
    eval_period: int = 10000
    aug_kwargs: Dict = field(default_factory=lambda: {})

    def update(self, param_dict: Dict) -> "Flags":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self

<a id="pred_scripts"></a>
### Prediction scripts

Now the methods are ready. Main training scripts starts from here.

In [ ]:
inputdir = Path("/kaggle/input/")
# traineddir = inputdir / "covid19-detectron2-models/RN101/results/RN101_with_negative_False"
traineddir = inputdir / "covid19-detectron2-models/RN101-full/RN101-full"

# flags = Flags()
flags: Flags = Flags().update(load_yaml(str(traineddir/"flags.yaml")))
print("flags", flags)
debug = flags.debug
# flags_dict = dataclasses.asdict(flags)
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)

# --- Read data ---
datadir = Path("/kaggle/working/") 
imgdir = inputdir / "test_images_512"

In [ ]:
cfg = get_cfg()
original_output_dir = cfg.OUTPUT_DIR
cfg.OUTPUT_DIR = str(outdir)
print(f"cfg.OUTPUT_DIR {original_output_dir} -> {cfg.OUTPUT_DIR}")

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("siim_covid19_train",)
cfg.DATASETS.TEST = ()
# cfg.DATASETS.TEST = ("vinbigdata_train",)
# cfg.TEST.EVAL_PERIOD = 50
cfg.DATALOADER.NUM_WORKERS = 2
# Let training initialize from model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = flags.base_lr  # pick a good LR
cfg.SOLVER.MAX_ITER = flags.iter
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = flags.roi_batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

### --- Inference & Evaluation ---
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
# path to the model we just trained
cfg.MODEL.WEIGHTS = str(traineddir/"model_final.pth")
print("Original thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)  # 0.05
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0  # set a custom testing threshold
print("Changed  thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
predictor = DefaultPredictor(cfg)

In [ ]:
DatasetCatalog.register("siim_covid19_test", lambda: get_test_datasetdicts(test_image_files))
MetadataCatalog.get("siim_covid19_test").set(thing_classes=thing_classes)
metadata = MetadataCatalog.get("siim_covid19_test")

In [ ]:
dataset_dicts = get_test_datasetdicts(test_image_ids)
dataset_dicts[:3]

In [ ]:
results_list = []
index = 0
batch_size = 4

boxes,scores,labels,dims = [],[],[],[]
boxes_tta,scores_tta,labels_tta,dims_tta = [],[],[],[]

for i in tqdm(range(ceil(len(dataset_dicts) / batch_size))):
    
    # predict batch
    inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
    dataset_dicts_batch = [dataset_dicts[i] for i in inds]
    im_list       = [cv2.imread(d["file_name"]) for d in dataset_dicts_batch]
    image_id_list = [d["image_id"] for d in dataset_dicts_batch]
    outputs_list, outputs_list_tta = predict_batch(predictor, im_list)
    
    for im, image_id, outputs, outputs_tta, d in zip(im_list, image_id_list, outputs_list, outputs_list_tta, dataset_dicts_batch):
        resized_height, resized_width, ch = im.shape
        
        # visualize few predictions
        if index < 5:
            # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
            v = Visualizer(
                im[:, :, ::-1],
                metadata=metadata,
                scale=0.5,
                instance_mode=ColorMode.IMAGE_BW
                # remove the colors of unsegmented pixels. This option is only available for segmentation models
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            cv2.imwrite(str(outdir / f"pred_{index}.jpg"), out.get_image()[:, :, ::-1])

        dim0, dim1 = imageid2size[image_id]
        
        # post-process
        instances = outputs["instances"]
        instances_tta = outputs_tta["instances"]

        if len(instances) == 0:
            # Negative/No Finding, let's set none 1 0 0 1 1.
            result = {"image_id": image_id, "PredictionString": "none 1 0 0 1 1"}
            
        else:
            # Find some bbox...
            # print(f"index={index}, find {len(instances)} bbox.")
            fields = instances.get_fields()
            fields_tta = instances_tta.get_fields()
                
            pred_classes = fields["pred_classes"]  # (n_boxes,)
            pred_scores = fields["scores"]
            pred_boxes = fields["pred_boxes"].tensor # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
            
            pred_classes_tta = fields_tta["pred_classes"]  # (n_boxes,)
            pred_scores_tta = fields_tta["scores"]
            pred_boxes_tta = fields_tta["pred_boxes"].tensor # shape (n_boxes, 4). (xmin, ymin, xmax, ymax)
            
            h_ratio = dim0 / resized_height
            w_ratio = dim1 / resized_width
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio
            pred_boxes_tta[:, [0, 2]] *= w_ratio
            pred_boxes_tta[:, [1, 3]] *= h_ratio            
            # revert flip tta
            pred_boxes_tta[:, [0, 2]] = (dim1 - pred_boxes_tta[:, [0, 2]]).flip(dims=[1]) 
            
            
            pred_classes_array = pred_classes.cpu().numpy()
            pred_boxes_array = pred_boxes.cpu().numpy()
            pred_scores_array = pred_scores.cpu().numpy()
            
            pred_classes_tta_array = pred_classes_tta.cpu().numpy()
            pred_boxes_tta_array = pred_boxes_tta.cpu().numpy()
            pred_scores_tta_array = pred_scores_tta.cpu().numpy()
            
            pred_boxes_normalized_array = pred_boxes_array.copy()
            pred_boxes_normalized_tta_array = pred_boxes_tta_array.copy()
            pred_boxes_normalized_array[:, [0, 2]] /= dim1
            pred_boxes_normalized_array[:, [1, 3]] /= dim0
            pred_boxes_normalized_tta_array[:, [0, 2]] /= dim1
            pred_boxes_normalized_tta_array[:, [1, 3]] /= dim0
            
            # append results for ensembling
            boxes.append(pred_boxes_normalized_array)
            scores.append(pred_scores_array)
            labels.append(pred_classes_array)
            dims.append([dim0, dim1])

            boxes_tta.append(pred_boxes_normalized_tta_array)
            scores_tta.append(pred_scores_tta_array)
            labels_tta.append(pred_classes_tta_array)
            dims_tta.append([dim0, dim1])
            
            result = {
                "id": image_id,
                "PredictionString": format_pred(
                    pred_classes_array, pred_boxes_array, pred_scores_array
                ),
            }
        
        # append result for image
        results_list.append(result)
        index += 1

In [ ]:
# all image detections before post-processing
image_sub_df = pd.DataFrame(results_list, columns=['id', 'PredictionString'])
image_sub_df['id'] = image_sub_df['id']+"_image"

In [ ]:
image_sub_df.shape

In [ ]:
image_sub_df.head()

In [ ]:
pred_images = get_image_files("/kaggle/working/results/")
show_images(L(map(PILImage.create, pred_images)),nrows=1,figsize=(50,10))

In [ ]:
gc.collect()
torch.cuda.empty_cache()

#### Weighted Boxes Fusion (TTA)

In [ ]:
sys.path.append("../input/weightedboxesfusion/")

In [ ]:
from ensemble_boxes import *

In [ ]:
fusion_prediction_string = []
detectron_boxes = []
detectron_scores = []
detectron_labels = []

for i in progress_bar(range((len(boxes)))):
    fusion_boxes, fusion_scores, fusion_labels = weighted_boxes_fusion([boxes[i], boxes_tta[i]],
                                                                       [scores[i], scores_tta[i]],
                                                                       [labels[i], labels_tta[i]], iou_thr=0.6, skip_box_thr=0)
    detectron_boxes.append(fusion_boxes.copy())
    detectron_scores.append(fusion_scores.copy())
    detectron_labels.append(fusion_labels.copy())
    
    fusion_boxes[:, [0, 2]] *= dims[i][1]
    fusion_boxes[:, [1, 3]] *= dims[i][0]
    fusion_prediction_string.append(format_pred(fusion_labels, fusion_boxes, fusion_scores))

In [ ]:
image_sub_df['PredictionString'] = fusion_prediction_string

In [ ]:
image_sub_df.head()

In [ ]:
assert not image_sub_df.isna().any().any()

# Detection (YOLO-V5)

https://www.kaggle.com/h053473666/siim-cov19-yolov5-train (opacity and none classes trained together)

In [ ]:
yolo_weights_dir = "/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt"
imgdir = "/kaggle/working/test_images_512/"

In [ ]:
!python /kaggle/input/yolov5-official-v31-dataset/yolov5/detect.py --weights {yolo_weights_dir}\
--img 512\
--conf 0.001\
--iou 0.5\
--source {imgdir}\
--save-txt --save-conf --exist-ok 

In [ ]:
image_ids = image_sub_df['id'].values

In [ ]:
len(detectron_boxes), len(detectron_scores), len(detectron_labels), len(dims), len(image_ids)

In [ ]:
fusion_prediction_string = []
for i in progress_bar(range(len(image_ids))):

    # read and convert to xmin,ymin,xmax,ymax
    pred_path = Path(f"/kaggle/working/runs/detect/exp/labels/{image_ids[i][:-6]}.txt")
    if pred_path.exists():
        pred = np.loadtxt(pred_path).reshape(-1,6)
        yolo_labels, yolo_scores, yolo_boxes = pred[:,0], pred[:,5], pred[:,1:5]
        yolo_boxes[..., [0, 1]] = yolo_boxes[..., [0, 1]] - yolo_boxes[..., [2, 3]]/2
        yolo_boxes[..., [2, 3]] = yolo_boxes[..., [0, 1]] + yolo_boxes[..., [2, 3]]

        # weighted boxes fusion
        fusion_boxes, fusion_scores, fusion_labels = weighted_boxes_fusion([detectron_boxes[i], yolo_boxes],
                                                                           [detectron_scores[i], yolo_scores],
                                                                           [detectron_labels[i], yolo_labels], iou_thr=0.6, skip_box_thr=0)
    else:
        fusion_boxes, fusion_scores, fusion_labels = detectron_boxes[i], detectron_scores[i], detectron_labels[i]
    
    fusion_boxes[:, [0, 2]] *= dims[i][1]
    fusion_boxes[:, [1, 3]] *= dims[i][0]
    fusion_prediction_string.append(format_pred(fusion_labels, fusion_boxes, fusion_scores))

In [ ]:
len(fusion_prediction_string)

In [ ]:
image_sub_df['PredictionString'] = fusion_prediction_string

In [ ]:
image_sub_df.head()

In [ ]:
assert not image_sub_df.isna().any().any()

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Classification

In [ ]:
from kornia.augmentation import augmentation as korniatfm
import torchvision.transforms as tvtfm
import kornia
from PIL import ImageOps
class RandomGaussianBlur(RandTransform):
    "Randomly apply gaussian blur with probability `p` with a value of s"
    order = 11
    def __init__(self, p=0.5, s=(8,32), same_on_batch=False, **kwargs): 
        store_attr()
        super().__init__(p=p, **kwargs)
        
    def encodes(self, x:TensorImage):
        if isinstance(self.s, tuple): s = np.random.randint(*self.s)
        if isinstance(self.s, list):  s = np.random.randint(*self.s)
        if isinstance(self.s, int):   s = self.s
        s2 = int(s/4)*2+1
        tfm = korniatfm.GaussianBlur((s2,s2),(s,s),same_on_batch=self.same_on_batch,p=1.)
        return tfm(x)

@delegates(to=korniatfm.RandomAffine)
class RandomAffineGeometric(RandTransform):
    "Randomly apply rotation,translation,scale, with probability `p`"
    order = 11
    def __init__(self, p=0.5, **kwargs): 
        store_attr()
        super().__init__(p=p)
        self.tfm = korniatfm.RandomAffine(p=1, **kwargs)
        
    def encodes(self, x:TensorImage): return self.tfm(x)
    
class RandHistEqualize(RandTransform):
    "Randomly flip with probability `p`"
    def __init__(self, p=0.5): super().__init__(p=p)
    def encodes(self, x:(Image.Image)): return ImageOps.equalize(x)

In [ ]:
sys.path.append("/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master/")
import timm
from torch.utils.checkpoint import checkpoint, checkpoint_sequential

# from: https://github.com/KeremTurgutlu/self_supervised/blob/main/nbs/02%20-%20layers.ipynb

mk_class('PoolingType', **{o:o.lower() for o in ['Fast', 'Avg', 'AvgMax', 'CatAvgMax', 'Max']},
         doc="All possible pooling types as attributes to get tab-completion and typo-proofing")

def create_fastai_encoder(arch:str, pretrained=False, n_in=3, pool_type=PoolingType.CatAvgMax):
    "Create timm encoder from a given arch backbone"
    encoder = create_body(arch, n_in, pretrained, cut=None)
    pool = AdaptiveConcatPool2d() if pool_type == "catavgmax" else nn.AdaptiveAvgPool2d(1)
    return nn.Sequential(*encoder, pool, Flatten())

def create_timm_encoder(arch:str, pretrained=False, n_in=3, pool_type=PoolingType.CatAvgMax):
    "Creates a body from any model in the `timm` library. If pool_type is None then it uses timm default"
    if ('vit' in arch) or (pool_type is None):
        model = timm.create_model(arch, pretrained=pretrained, in_chans=n_in, num_classes=0)
    else:
        model = timm.create_model(arch, pretrained=pretrained, in_chans=n_in, num_classes=0, global_pool=pool_type)
    return model

def create_encoder(arch:str, pretrained=True, n_in=3, pool_type=PoolingType.CatAvgMax):
    "A utility for creating encoder without specifying the package"
    if arch in globals(): return create_fastai_encoder(globals()[arch], pretrained, n_in, pool_type)
    else:                 return create_timm_encoder(arch, pretrained, n_in, pool_type)

def create_cls_module(nf, n_out, lin_ftrs=None, ps=0.5, use_bn=True, first_bn=True, bn_final=False, lin_first=False, y_range=None):
    "Creates classification layer which takes nf flatten features and outputs n_out logits"
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
    bns = [first_bn] + [use_bn]*len(lin_ftrs[1:])
    ps = L(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    if lin_first: layers.append(nn.Dropout(ps.pop(0)))
    for ni,no,bn,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
        layers += LinBnDrop(ni, no, bn=bn, p=p, act=actn, lin_first=lin_first)
    if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    if y_range is not None: layers.append(SigmoidRange(*y_range))
    return nn.Sequential(*layers)

def create_model(arch, n_out, **kwargs):
    encoder = create_encoder(arch, **kwargs)
    with torch.no_grad(): out = encoder(torch.randn((2,3,384,384))) 
    classifier = create_cls_module(out.size(-1), n_out, **kwargs)
    return nn.Sequential(encoder, classifier)

def model_splitter(m): return L(m[0], m[1]).map(params)


class CheckpointResNet(Module):
    def __init__(self, resnet_model, checkpoint_nchunks=2):
        "A gradient checkpoint wrapper for timm ResNet"
        self.checkpoint_nchunks = checkpoint_nchunks
        self.resnet_model = resnet_model
        self.forward_layers = nn.Sequential(*[
            self.resnet_model.layer1,
            self.resnet_model.layer2,
            self.resnet_model.layer3,
            self.resnet_model.layer4
        ])
    
    def forward(self, x):
        x = self.resnet_model.conv1(x)
        x = self.resnet_model.bn1(x)
        x = self.resnet_model.act1(x)
        x = self.resnet_model.maxpool(x)
            
        x = checkpoint_sequential(self.forward_layers, self.checkpoint_nchunks, x)
        x = self.resnet_model.global_pool(x)
        
        if self.resnet_model.drop_rate:
            x = F.dropout(x, p=float(self.resnet_model.drop_rate), training=self.resnet_model.training)
        x = self.resnet_model.fc(x)
        return x
    
class CheckpointEfficientNet(Module):
    def __init__(self, effnet_model, checkpoint_nchunks=2):
        "A gradient checkpoint wrapper for timm EfficientNet"
        self.checkpoint_nchunks = checkpoint_nchunks
        self.effnet_model = effnet_model
    
    def forward_features(self, x):
        x = self.effnet_model.conv_stem(x)
        x = self.effnet_model.bn1(x)
        x = self.effnet_model.act1(x)
        x = checkpoint_sequential(self.effnet_model.blocks, self.checkpoint_nchunks, x)
        x = self.effnet_model.conv_head(x)
        x = self.effnet_model.bn2(x)
        x = self.effnet_model.act2(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.effnet_model.global_pool(x)
        if self.effnet_model.drop_rate > 0.:
            x = F.dropout(x, p=self.effnet_model.drop_rate, training=self.effnet_model.training)
        return self.effnet_model.classifier(x)
    

class CheckpointVisionTransformer(Module):
    def __init__(self, vit_model, checkpoint_nchunks=2):
        "A gradient checkpoint wrapper for timm VisionTransformer"
        self.checkpoint_nchunks = checkpoint_nchunks
        self.vit_model = vit_model
    
    def forward_features(self, x):
        B = x.shape[0]
        x = self.vit_model.patch_embed(x)

        cls_tokens = self.vit_model.cls_token.expand(B, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.vit_model.pos_embed
        x = self.vit_model.pos_drop(x)
        x = checkpoint_sequential(self.vit_model.blocks, self.checkpoint_nchunks, x)
        x = self.vit_model.norm(x)[:, 0]
        x = self.vit_model.pre_logits(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.vit_model.head(x)
        return x
    
    
class CheckpointNFNet(Module):
    def __init__(self, nfnet_model, checkpoint_nchunks=2):
        "A gradient checkpoint wrapper for timm NFNet"
        self.checkpoint_nchunks = checkpoint_nchunks
        self.nfnet_model = nfnet_model
    
    def forward_features(self, x):
        x = self.nfnet_model.stem(x)
        x = checkpoint_sequential(self.nfnet_model.stages, self.checkpoint_nchunks, x)
        x = self.nfnet_model.final_conv(x)
        x = self.nfnet_model.final_act(x)
        return x
    
    def forward(self, x):
        x = self.forward_features(x)
        x = self.nfnet_model.head(x)
        return x


def get_classification_model(arch, n_out=4):
    
    if "res" in arch:
        if "200d" in arch:
            encoder = CheckpointResNet(create_timm_encoder(arch), checkpoint_nchunks=4)    
        else:
            encoder = CheckpointResNet(create_timm_encoder(arch), checkpoint_nchunks=2)
        
    elif ("eff" in arch) and (arch != "tf_efficientnet_b0_ns") :
        if ("b7" in arch or "l2" in arch):
            encoder = CheckpointEfficientNet(create_timm_encoder(arch), checkpoint_nchunks=4)
        else:
            encoder = CheckpointEfficientNet(create_timm_encoder(arch), checkpoint_nchunks=2)
            
    elif ("vit" in arch):
        if ("base" in arch):
            encoder = CheckpointVisionTransformer(create_timm_encoder(arch), checkpoint_nchunks=2)
        elif ("large" in arch):
            encoder = CheckpointVisionTransformer(create_timm_encoder(arch), checkpoint_nchunks=8)
        else:
            encoder = CheckpointVisionTransformer(create_timm_encoder(arch), checkpoint_nchunks=4)
            
    elif ("nfnet" in arch):
        if ("f2" in arch or "f3" in arch or "f4" in arch or "f5" in arch):
            encoder = CheckpointNFNet(create_timm_encoder(arch), checkpoint_nchunks=4)
        
    elif ("cait" in arch):
        encoder = CheckpointVisionTransformer(create_timm_encoder(arch), checkpoint_nchunks=6)
            
    else:
        encoder = create_timm_encoder(arch)
        
    with torch.no_grad(): out = encoder(torch.randn((2,3,384,384))) 
    classifier = create_cls_module(out.size(-1), n_out=n_out)
    model = nn.Sequential(encoder, classifier)
    return model

In [ ]:
sys.path.append("/kaggle/input/effdet024/effdet-0.2.4/")
import effdet
from effdet.efficientdet import *
from effdet.config.model_config import *

def get_efficientdet_config(model_name='tf_efficientdet_d7'):
    """Get the default config for EfficientDet based on model name."""
    h = default_detection_model_configs()
    h.update(efficientdet_model_param_dict[model_name])
    h.num_levels = h.max_level - h.min_level + 1
    h = deepcopy(h)  # may be unnecessary, ensure no references to param dict values
    # OmegaConf.set_struct(h, True)  # FIXME good idea?
    return h

class CheckpointEfficientNetFeatures(Module):
    def __init__(self, backbone):
        
        self.backbone = backbone
        
    def forward(self, x):
        x = self.backbone.conv_stem(x)
        x = self.backbone.bn1(x)
        x = self.backbone.act1(x)
        if self.backbone.feature_hooks is None:
            features = []
            if 0 in self.backbone._stage_out_idx:
                features.append(x)  # add stem out
            for i, b in enumerate(self.backbone.blocks):
#                 x = b(x)
                x = checkpoint(b,x)
                if i + 1 in self.backbone._stage_out_idx:
                    features.append(x)
            return features
        else:
#             self.blocks(x)
            checkpoint_sequential(self.backbone.blocks,7,x)
            out = self.backbone.feature_hooks.get_output(x.device)
            return list(out.values())
        
class EffDetAuxModel(Module):
    def __init__(self, model):
        self.model = model
        self.backbone = CheckpointEfficientNetFeatures(model.backbone)
        self.pooling = nn.Sequential(AdaptiveConcatPool2d(), nn.Flatten())
        
        nf = list(model.backbone.blocks[-1][-1].children())[-1].num_features
        self.cls_head = create_cls_module(nf*2, n_out=4)

        nc = list(list(model.class_net.children())[-1].children())[-1].out_channels
        self.conv1x1 = nn.Conv2d(nc,1,1)
        
    def forward(self, x):
        # effnet classification 
        x = self.backbone(x)
        x_pooled = self.pooling(x[-1]) # last block output
        cls_out = self.cls_head(x_pooled)

        # effdet segmentation (output resolution: image_size/2^2)
        x = self.model.fpn(x)
        x = self.model.class_net(x)
        seg_out = self.conv1x1(x[0]) # top bifpn output
        
        return cls_out, seg_out
    
def create_aux_model(arch='tf_efficientnet_b7_ns'):
    config = get_efficientdet_config()
    config['backbone_name'] = arch
    config['backbone_indices'] = (1,2,3,4)
    config['min_level'] = 2
    config['max_level'] = 7
    config['num_levels'] = 6
    config['url'] = ''
    config['fpn_channels'] = 64
    config['fpn_cell_repeats'] = 3
    config['box_class_repeats'] = 3
    model = EfficientDet(config, pretrained_backbone=False)
    return EffDetAuxModel(model)

class ClassificationWrapper(Module):
    def __init__(self, model):
        self.model = model
        
    def forward(self, x):
        x = self.model.backbone(x)
        x_pooled = self.model.pooling(x[-1]) # last block output
        return self.model.cls_head(x_pooled)

In [ ]:
def read_image(image_id):  return PILImage.create(f"/kaggle/working/test_images_512/{image_id}.png")
def dummy_label(image_id): return 0

# TEST DL
source = test_image_ids
tfms = [[read_image, ToTensor], [dummy_label]]
dsets = Datasets(source, tfms=tfms, splits=None)
batch_tfms = [IntToFloatTensor] 
dls = dsets.dataloaders(bs=128, num_workers=None, after_batch=batch_tfms)
test_dl_512 = dls.test_dl(test_image_ids, bs=16)

In [ ]:
# def read_image(image_id):  return PILImage.create(f"/kaggle/working/test_images_384/{image_id}.png")
# def dummy_label(image_id): return 0

# # TEST DL
# source = test_image_ids
# tfms = [[read_image, ToTensor], [dummy_label]]
# dsets = Datasets(source, tfms=tfms, splits=None)
# batch_tfms = [IntToFloatTensor] 
# dls = dsets.dataloaders(bs=128, num_workers=None, after_batch=batch_tfms)
# test_dl_384 = dls.test_dl(test_image_ids, bs=16)

In [ ]:
# TTA
rotate = Rotate(p=1., max_deg=25, pad_mode='zeros')
zoom = Zoom(p=1., min_zoom=0.75, max_zoom=1.35, pad_mode='zeros')
normalize = Normalize.from_stats(*imagenet_stats)

In [ ]:
torch.cuda.empty_cache()

#### Predictions

In [ ]:
import gc

In [ ]:
all_models_512 = []
all_models_384 = []
models_dir = Path("/kaggle/input/covid19-classification-models/")
models_dir2 = Path("/kaggle/input/covid19-classification-models2/")

In [ ]:
# for fold_idx in range(8):
#     use_norm = False
#     model = create_aux_model("tf_efficientnet_b7_ns")
#     state_dict = torch.load(models_dir/f"tf_efficientnet_b7-sz384-ricord-nih_pretrained-aux_seg_loss0.75-fold{fold_idx}.pth") # sigmoid - no normalization
#     model.load_state_dict(state_dict)
#     model = ClassificationWrapper(model)
#     all_models_384.append((model, use_norm)) # model, use_norm

In [ ]:
for fold_idx in range(8):
    use_norm = False
    model = create_aux_model("tf_efficientnet_b7_ns")
    state_dict = torch.load(models_dir/f"tf_efficientnet_b7-sz512-ricord-aux_seg_loss0.75-fold{fold_idx}.pth") # sigmoid - no normalization
    model.load_state_dict(state_dict)
    model = ClassificationWrapper(model)
    all_models_512.append((model, use_norm)) # model, use_norm

In [ ]:
for fold_idx in range(8):
    use_norm = False
    model = create_aux_model("tf_efficientnet_b7_ns")
    state_dict = torch.load(models_dir/f"tf_efficientnet_b7_ns-sz512-ricord-aux_seg_loss0.5-fold{fold_idx}.pth") # sigmoid - no normalization
    model.load_state_dict(state_dict)
    model = ClassificationWrapper(model)
    all_models_512.append((model, use_norm)) # model, use_norm

In [ ]:
# use_norm = False
# model = create_aux_model("tf_efficientnet_b7_ns")
# state_dict = torch.load(models_dir2/f"tf_efficientnet_b7-sz512-ricord-nih_pretrained-aux_seg_loss0.5-foldfull_data.pth") # sigmoid - no normalization
# model.load_state_dict(state_dict)
# model = ClassificationWrapper(model)
# all_models_512.append((model, use_norm)) # model, use_norm

In [ ]:
for fold_idx in range(5):
    use_norm = True
    model = get_classification_model("tf_efficientnet_b7_ns")
    state_dict = torch.load(models_dir/f"tf_efficientnet_b7_ns-re-mixup-fold{fold_idx}.pth") # softmax
    model.load_state_dict(state_dict)
    all_models_512.append((model, use_norm))

In [ ]:
len(all_models_512), len(all_models_384)

In [ ]:
# all_preds_384 = []

# # TEST PREDS (384)
# with torch.no_grad():
#     all_models_384 = [(model.eval().cuda(), use_norm) for model, use_norm in all_models_384]

#     for xb, in progress_bar(test_dl_384):
#         tta_preds = []
#         for model, use_norm in all_models_384:
#             # original
#             out1 = to_detach(model(normalize(xb) if use_norm else xb))
#             # flip lr
#             out2 = to_detach(model(normalize(xb.flip_lr()) if use_norm else xb.flip_lr()))
#             # rotate
#             out3 = to_detach(model(normalize(rotate(xb)) if use_norm else rotate(xb)))
#             # zoom
#             out4 = to_detach(model(normalize(zoom(xb)) if use_norm else zoom(xb)))
            
#             tta_preds += [torch.stack([out1, out2, out3, out4]).permute(1,0,2)]
#         tta_preds = torch.stack(tta_preds).permute(1,0,2,3) # bs x models x n_tta x n_classes
#         all_preds_384 += [tta_preds]

        
all_preds_512 = []
# TEST PREDS (512)
with torch.no_grad():
    all_models_512 = [(model.eval().cuda(), use_norm) for model, use_norm in all_models_512]

    for xb, in progress_bar(test_dl_512):
        tta_preds = []
        for model, use_norm in all_models_512:
            # original
            out1 = to_detach(model(normalize(xb) if use_norm else xb))
            # flip lr
            out2 = to_detach(model(normalize(xb.flip_lr()) if use_norm else xb.flip_lr()))
            # rotate
            out3 = to_detach(model(normalize(rotate(xb)) if use_norm else rotate(xb)))
            # zoom
            out4 = to_detach(model(normalize(zoom(xb)) if use_norm else zoom(xb)))
            
            tta_preds += [torch.stack([out1, out2, out3, out4]).permute(1,0,2)]
#             tta_preds += [torch.stack([out1, out2, out3]).permute(1,0,2)]
        tta_preds = torch.stack(tta_preds).permute(1,0,2,3) # bs x models x n_tta x n_classes
        all_preds_512 += [tta_preds]

In [ ]:
# # single sigmoid model
# all_preds = torch.cat(all_preds).sigmoid(); all_preds.shape
# mean_preds = all_preds.mean((1,2))

In [ ]:
# multi model power average
# all_preds_384 = torch.cat(all_preds_384)
all_preds_512 = torch.cat(all_preds_512)

# preds1 = all_preds_384.sigmoid().mean((1,2))
preds1 = all_preds_512[:,:8].sigmoid().mean((1,2))
preds2 = all_preds_512[:,8:16].sigmoid().mean((1,2))
preds3 = all_preds_512[:,16:21].softmax(-1).mean((1,2))

p = 2
mean_preds = (preds1**p)*0.4 + (preds2**p)*0.4 + (preds3**p)*0.2

In [ ]:
def generate_study_submission(test_image_ids, preds, image_id2study_id):
    "Aggregate image preds for each study, take mean if multiple, return submission df"
    study_preds_dict = defaultdict(list)
    for img_id, probas in zip(test_image_ids, to_np(preds)):
        sid = image_id2study_id[img_id]
        study_preds_dict[sid].append(probas)
    
    study_classes = ["negative", "typical", "indeterminate", "atypical"]
    row1,row2 = [],[]
    for sid, probas in study_preds_dict.items(): 
        row1.append(f"{sid}_study")
        probas = np.vstack(probas).mean(0) # take mean if multiple images per study
        s = []
        for class_id,proba in enumerate(probas):
            s += [f"{study_classes[class_id]} {proba} 0 0 1 1"]
        row2.append(" ".join(s))
    study_sub_df = pd.DataFrame()
    study_sub_df['id'] = row1
    study_sub_df['PredictionString'] = row2
    return study_sub_df

In [ ]:
study_sub_df = generate_study_submission(test_image_ids, mean_preds, image_id2study_id); study_sub_df.shape

In [ ]:
study_sub_df.head()

In [ ]:
image_sub_df.head()

### Post-Process

In [ ]:
# add none to all opacity preds CV: 0.682 * 1/3 = 0.227
none_probas = mean_preds[:,0].numpy()
image_sub_postproc = image_sub_df.copy()
rows = []
for p,s in zip(none_probas, image_sub_postproc['PredictionString']):
    rows.append(f'none {p} 0 0 1 1' + ' ' + s)
image_sub_postproc['PredictionString'] = rows

In [ ]:
image_sub_df = image_sub_postproc

In [ ]:
# # set negative predictions as none with their probas CV: 0.597 * 1/3 = 0.198
# max_vals_idxs = mean_preds.max(dim=-1)
# max_vals = max_vals_idxs.values[max_vals_idxs.indices == 0].numpy()
# none_prediction_string = [f'none {proba} 0 0 1 1' for proba in max_vals]
# image_sub_df.loc[(max_vals_idxs.indices == 0).numpy(), 'PredictionString'] = none_prediction_string

# Submit

In [ ]:
sub_df = pd.concat([study_sub_df,image_sub_df])
sub_df.to_csv("submission.csv",index=False)
!rm -r /kaggle/working/test_images*
!rm -r /kaggle/working/results
!rm -r /kaggle/working/runs

In [ ]:
sub_df

### fin